# Introduction

In [2]:
from ipywebcam import WebCamWidget

In [3]:
w = WebCamWidget(constraints={"audio": True, "video": True})

In [4]:
w.video_codec_selector

Dropdown(description='Video codec', options=('video/VP8', 'video/H264 level-asymmetry-allowed=1;packetization-…

In [5]:
w.audio_input_selector

Dropdown(description='Audio input device', options=(('默认 - 麦克风 (VIA HD Audio(Win 10))', {'deviceId': 'default'…

In [6]:
w

WebCamWidget(audio_input_device={'deviceId': 'default', 'kind': 'audioinput', 'label': '默认 - 麦克风 (VIA HD Audio…

In [7]:
w.audio_input_selector

Dropdown(description='Audio input device', options=(('默认 - 麦克风 (VIA HD Audio(Win 10))', {'deviceId': 'default'…

In [8]:
w.audio_input_devices

[{'deviceId': 'default',
  'kind': 'audioinput',
  'label': '默认 - 麦克风 (VIA HD Audio(Win 10))',
  'groupId': 'ba58835e0d4c7966dfec1fd3aba0d520b31ab72e8c853e8e36e8ce23e57edfa4'},
 {'deviceId': 'communications',
  'kind': 'audioinput',
  'label': '通讯 - 麦克风 (VIA HD Audio(Win 10))',
  'groupId': 'ba58835e0d4c7966dfec1fd3aba0d520b31ab72e8c853e8e36e8ce23e57edfa4'},
 {'deviceId': 'd59961d05ec0043af5559a09a38f93eefcac77a6df29159191b82e62ab6cf2ea',
  'kind': 'audioinput',
  'label': '麦克风 (VIA HD Audio(Win 10))',
  'groupId': 'ba58835e0d4c7966dfec1fd3aba0d520b31ab72e8c853e8e36e8ce23e57edfa4'}]

In [9]:
w.audio_input_device

{'deviceId': 'default',
 'kind': 'audioinput',
 'label': '默认 - 麦克风 (VIA HD Audio(Win 10))',
 'groupId': 'ba58835e0d4c7966dfec1fd3aba0d520b31ab72e8c853e8e36e8ce23e57edfa4'}

In [13]:
import cv2
from av import VideoFrame
async def transform(frame, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    new_frame.pts = frame.pts
    new_frame.time_base = frame.time_base
    return new_frame

transformer = w.add_video_transformer(transform)

In [12]:
w.remove_video_transformer(transformer)